In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
from verification.val_db import (get_finalized_validation_datasets,
                                 read_validation_dataset,
                                 get_val_s3_path
                                )

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

import numpy as np
from tqdm import tqdm
import rasterio
from matplotlib.colors import ListedColormap
from shapely.geometry import box
from rasterio.plot import show
from pathlib import Path
from verification.rio_tools import get_geopandas_features_from_array
import pandas as pd
import sklearn.metrics
import json
from verification.hls import get_hls_urls

This index is just relative to the finalized datasets.

In [ ]:
PLANET_ID = '20211002_155415_1009'

# Table of Finalized Data

In [ ]:
df = get_finalized_validation_datasets()
df.head()

In [ ]:
f'Currently, there are {df.shape[0]} finalized datasets'

# Read a Validation Dataset

In [ ]:
X_val, p_val = read_validation_dataset(PLANET_ID)
p_val

In [ ]:
Y = X_val.astype(float)
Y[Y == 255] = np.nan

fontSize=10
fig, ax=plt.subplots(figsize=(7,7))
cmap = ListedColormap(['white', 'blue'])
im=ax.imshow(Y, interpolation='none',cmap=cmap,vmin=0,vmax=1)
show(Y, interpolation='none',cmap=cmap,vmin=0,vmax=1,transform=p_val['transform'],ax=ax)
cbar = fig.colorbar(im, ax=ax,shrink=0.7,ticks=[0.25, 0.75])
cbar.ax.set_yticklabels(['Not Water', 'Open Water']);  # vertically oriented colorbar
plt.xlabel('UTM easting (meters)',fontsize=fontSize)
plt.ylabel('UTM northing (meters)',fontsize=fontSize)
plt.xticks(fontsize = fontSize)
plt.yticks(fontsize = fontSize)
ax.set_title('Validation Dataset');
ax.ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax.yaxis.get_offset_text().set_fontsize(fontSize)
ax.xaxis.get_offset_text().set_fontsize(fontSize)

# Load Local Validation Table

In [ ]:
df_validation_table = pd.read_csv('validation_table_data.csv')
df_validation_table.head()

# Get Associated HLS Id

In [ ]:
HLS_ID = df_validation_table[df_validation_table.planet_id == PLANET_ID].hls_id.iloc[0]
HLS_ID

# Get DSWx Products

In [ ]:
dswx_urls = df_validation_table[df_validation_table.planet_id == PLANET_ID].dswx_urls.iloc[0].split(',')
dswx_urls

In [ ]:
with rasterio.open(dswx_urls[0]) as ds:
    X_dswx = ds.read(1)
    p_dswx = ds.profile
    crs_dswx = ds.crs
    colormap = ds.colormap(1)
    dswx_crs = ds.crs

In [ ]:
fontSize=7
fontSizeTitle=8
fig, ax = plt.subplots(dpi=150,figsize=(5,5))
cmap = ListedColormap([np.array(colormap[key]) / 255 for key in range(256)])
im_dswx=ax.imshow(X_dswx, interpolation='none',cmap=cmap,vmin=0,vmax=255)
show(X_dswx, interpolation='none',cmap=cmap,vmin=0,vmax=255,transform=p_dswx['transform'],ax=ax)
cbar=fig.colorbar(im_dswx,ax=ax,shrink=0.4,ticks=[0.5, 1.5, 2.5,5.5,8.5, 9.5])
cbar.set_ticklabels(['Not Water', 'Open Water','Partial Surface Water','Reserved','Snow/Ice','Cloud/Cloud Shadow'],fontsize=fontSize)   
cbar.ax.set_ylim(0,10)
plt.xlabel('UTM easting (meters)',fontsize=fontSize)
plt.ylabel('UTM northing (meters)',fontsize=fontSize)
plt.xticks(fontsize = fontSize)
plt.yticks(fontsize = fontSize)
ax.set_title('DSWx Product for HLS tile',fontsize=fontSizeTitle);
ax.ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax.yaxis.get_offset_text().set_fontsize(fontSize)
ax.xaxis.get_offset_text().set_fontsize(fontSize)

# Compare Extents

Inspect the DSWx Subset and it's corresponding Validation Dataset

In [ ]:
val_url = get_val_s3_path(PLANET_ID)
with rasterio.open(val_url) as ds:
    val_bounds = list(ds.bounds)
    val_crs = ds.crs

The two datasets are frequently in the same CRS. Just in case, we reproject to DSWx projection.

In [ ]:
df_val_bounds = gpd.GeoDataFrame(geometry=[box(*val_bounds).buffer(60)],
                                 crs=val_crs)
df_val_bounds = df_val_bounds.to_crs(dswx_crs)
df_val_bounds

In [ ]:
fontSize=7
fontSizeTitles=8
fig, ax = plt.subplots(1, 3, dpi=150, figsize=(10, 10))
cmap = ListedColormap([np.array(colormap[key]) / 255 for key in range(256)])

show(X_dswx, cmap=cmap, transform=p_dswx['transform'], vmin=0, vmax=255, interpolation='none', ax=ax[0])
df_val_bounds.to_crs(crs_dswx).boundary.plot(ax=ax[0], color='black')
ax[0].set_title('Full DSWx scene with val bbox',fontsize=fontSize)
ax[0].set_xlabel('UTM easting (meters)',fontsize=fontSizeTitles)
ax[0].set_ylabel('UTM northing (meters)',fontsize=fontSizeTitles)
ax[0].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax[0].tick_params(axis='both', which='major', labelsize=fontSize)
ax[0].yaxis.get_offset_text().set_fontsize(fontSize)
ax[0].xaxis.get_offset_text().set_fontsize(fontSize)

show(X_dswx, cmap=cmap, transform=p_dswx['transform'], vmin=0,vmax=255, interpolation='none', ax=ax[1])
val_bounds_dswx = df_val_bounds.to_crs(crs_dswx).total_bounds
ax[1].set_xlim(val_bounds_dswx[0], val_bounds_dswx[2])
ax[1].set_ylim(val_bounds_dswx[1], val_bounds_dswx[3])
ax[1].set_title('DSWx Subset Area',fontsize=fontSizeTitles)
ax[1].set_xlabel('UTM easting (meters)',fontsize=fontSize)
ax[1].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax[1].tick_params(axis='both', which='major', labelsize=fontSize)
ax[1].yaxis.get_offset_text().set_fontsize(fontSize)
ax[1].xaxis.get_offset_text().set_fontsize(fontSize)

show(X_val, transform=p_val['transform'], ax=ax[2], interpolation='none', cmap=cmap, vmin=0, vmax=255)
ax[2].set_title('Validation Dataset',fontsize=fontSizeTitles)
ax[2].set_xlabel('UTM easting (meters)',fontsize=fontSize)
ax[2].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax[2].tick_params(axis='both', which='major', labelsize=fontSize)
ax[2].yaxis.get_offset_text().set_fontsize(fontSize)
ax[2].xaxis.get_offset_text().set_fontsize(fontSize)

cbar=fig.colorbar(im_dswx,ax=ax,ticks=[0.5, 1.5, 2.5,5.5,8.5, 9.5],shrink=0.3,orientation='horizontal',fraction=0.32,pad=.05)
cbar.set_ticklabels(['Not Water', 'Open Water','Partial Surface Water','Reserved','Snow/Ice','Cloud/Cloud Shadow'],rotation=45)   
cbar.ax.tick_params(labelsize=6)
cbar.ax.set_xlim(0,10);

# Crop Datasets

In [ ]:
from dem_stitcher.rio_window import read_raster_from_window

In [ ]:
X_dswx_c, p_dswx_c = read_raster_from_window(dswx_urls[0], 
                                             list(df_val_bounds.total_bounds), 
                                             df_val_bounds.crs)

# Resample Validation Dataset to DSWx Product

This extracts a percent open surface water in HLS frame.

In [ ]:
from dem_stitcher.rio_tools import reproject_arr_to_match_profile, update_profile_resolution

X_val_temp = X_val.astype('float32')
X_val_temp[(X_val == 10) | (X_val == 255)] = np.nan

p_val_temp = p_val.copy()
p_val_temp['dtype'] = 'float32'
p_val_temp['nodata'] = np.nan

p_dswx_c_mod = update_profile_resolution(p_dswx_c, 
                                         p_val['transform'].a)


X_val_per_w_int, p_per_int = reproject_arr_to_match_profile(X_val_temp,
                                                    p_val_temp, 
                                                    p_dswx_c_mod)
X_val_per_w_int = X_val_per_w_int[0, ...]

X_val_per_w, _ = reproject_arr_to_match_profile(X_val_per_w_int,
                                                p_per_int, 
                                                p_dswx_c)
X_val_per_w = X_val_per_w[0, ...]

In [ ]:
fontSize=8
fig, ax = plt.subplots(dpi=150,figsize=(5, 5))
# cmap = ListedColormap([np.array(colormap[key]) / 255 for key in range(256)])
cmap = ListedColormap(['white', 'blue'])
im=ax.imshow(X_val_per_w, interpolation='none',cmap=cmap,vmin=np.nanmin(X_val_per_w),vmax=np.nanmax(X_val_per_w))
show(X_val_per_w, vmin=np.nanmin(X_val_per_w),vmax=np.nanmax(X_val_per_w), transform=p_dswx['transform'], ax=ax,cmap=cmap)
cbar=fig.colorbar(im,ax=ax,shrink=0.5,ticks=[0.25, 0.75])
cbar.set_ticklabels(['Not Water', 'Open Water'],fontsize=fontSize)   
cbar.ax.set_ylim(np.nanmin(X_val_per_w),np.nanmax(X_val_per_w))
ax.set_xlabel('UTM easting (meters)',fontsize=fontSize)
ax.set_ylabel('UTM northing (meters)',fontsize=fontSize)
plt.title('Validation data reprojected to DSWx grid',fontsize=fontSize)
plt.xticks(fontsize = fontSize)
plt.yticks(fontsize = fontSize);
ax.ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax.yaxis.get_offset_text().set_fontsize(fontSize)
ax.xaxis.get_offset_text().set_fontsize(fontSize)

Convert to DSWx Labels.

In [ ]:
X_val_r = np.full(X_val_per_w.shape, 255)

mask = np.isnan(X_val_per_w)

ind_w = (X_val_per_w == 1)
X_val_r[ind_w] = 1

ind_pw = (X_val_per_w >= .5) & (X_val_per_w < 1)
X_val_r[ind_pw] = 2

ind_nw = (X_val_per_w < .5)
X_val_r[ind_nw] = 0

In [ ]:
fontSize=6
fig, ax = plt.subplots(1, 2, dpi=150, figsize=(8, 8))
cmap = ListedColormap([np.array(colormap[key]) / 255 for key in range(256)])
im=ax[0].imshow(X_dswx_c, interpolation='none',cmap=cmap,vmin=0,vmax=255)

show(X_dswx_c, cmap=cmap, transform=p_dswx_c['transform'], interpolation='none', ax=ax[0], vmin=0,vmax=255)
ax[0].set_title('DSWx Subset Area',fontsize=8)
ax[0].set_xlabel('UTM easting (meters)',fontsize=fontSize)
ax[0].set_ylabel('UTM northing (meters)',fontsize=fontSize)
ax[0].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax[0].tick_params(axis='both', which='major', labelsize=fontSize)
ax[0].yaxis.get_offset_text().set_fontsize(fontSize)
ax[0].xaxis.get_offset_text().set_fontsize(fontSize)

show(X_val_r, cmap=cmap, transform=p_val['transform'], interpolation='none',ax=ax[1], vmin=0, vmax=255)
ax[1].set_title('Val. Data Reprojected and Relabeled',fontsize=8)
ax[1].set_xlabel('UTM easting (meters)',fontsize=fontSize)
ax[1].set_ylabel('UTM northing (meters)',fontsize=fontSize)

ax[1].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax[1].tick_params(axis='both', which='major', labelsize=fontSize)
ax[1].yaxis.get_offset_text().set_fontsize(fontSize)
ax[1].xaxis.get_offset_text().set_fontsize(fontSize)

cbar=fig.colorbar(im,ax=ax,ticks=[0.5, 1.5,2.5,5.5,8.5,9.5],shrink=0.5,orientation='horizontal',pad=.071)
cbar.set_ticklabels(['Not Water', 'Open Water','Partial Surface Water','Reserved','Snow/Ice','Cloud/Cloud Shadow'],rotation=45)   
cbar.ax.tick_params(labelsize=fontSize)
cbar.ax.set_xlim(0,10);

**Warning**: In many cases, the UTM zone of the validation dataset and the OPERA DSWx product do not match. So despite some of the side-by-side plots that appear parallel, the rotation above can come as surprise. A slight rotation will occur if the below CRS's are different.

In [ ]:
print('DSWx CRS: ', p_dswx['crs'])
print('Validation CRS: ', p_val['crs'])

# Save Relevant Rasters for Inspection

In [ ]:
dswx_urls[0].split('_B01')[0]

In [ ]:
DSWx_ID = dswx_urls[0].split('_B01')[0].split('/')[-1]
DSWx_ID

In [ ]:
out_dir = Path('verification_assessment_data') / DSWx_ID
out_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
p_val_r = p_dswx_c.copy()
p_val_r['dtype'] = np.uint8
p_val_r['nodata'] = 255

with rasterio.open(out_dir / f'validation_r_{DSWx_ID}.tif', 'w', **p_val_r) as ds:
    ds.write(X_val_r, 1)
    ds.write_colormap(1, colormap)

In [ ]:
p_perc_r = p_dswx_c.copy()
p_perc_r['dtype'] = np.float32
p_perc_r['nodata'] = np.nan

with rasterio.open(out_dir / f'percent_r_{DSWx_ID}.tif', 'w', **p_perc_r) as ds:
    ds.write(X_val_per_w, 1)

In [ ]:
with rasterio.open(out_dir / f'percent_intermediate_{DSWx_ID}.tif', 'w', **p_per_int) as ds:
    ds.write(X_val_per_w_int, 1)

In [ ]:
with rasterio.open(out_dir / f'{DSWx_ID}.tif', 'w', **p_dswx_c) as ds:
    ds.write(X_dswx_c, 1)
    ds.write_colormap(1, colormap)

In [ ]:
with rasterio.open(out_dir / f'validation_original_{DSWx_ID}.tif', 'w', **p_val) as ds:
    ds.write(X_val, 1)
    ds.write_colormap(1, colormap)

# Scene-wise stratified sampling

In [ ]:
shared_mask = (X_val_r == 255) | ~(np.isin(X_dswx_c, [0, 1, 2]))

fontSize=7
fig, ax = plt.subplots(dpi=150,figsize=(4, 4))
cmap = ListedColormap(['darkblue', 'yellow'])
im=ax.imshow(shared_mask, interpolation='none',cmap=cmap,vmin=np.nanmin(shared_mask),vmax=np.nanmax(shared_mask))
show(shared_mask, transform=p_val['transform'], interpolation='none', vmin=np.nanmin(shared_mask),vmax=np.nanmax(shared_mask),ax=ax,cmap=cmap)  
cbar=fig.colorbar(im,ax=ax,shrink=0.4,ticks=[0.25, .75])
cbar.set_ticklabels(['keep', 'remove'],fontsize=fontSize)   
cbar.ax.set_ylim(0,1)
ax.set_title('Shared mask',fontsize=8);
ax.ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax.yaxis.get_offset_text().set_fontsize(fontSize)
ax.xaxis.get_offset_text().set_fontsize(fontSize)
plt.xlabel('UTM easting (meters)',fontsize=fontSize)
plt.ylabel('UTM northing (meters)',fontsize=fontSize)
plt.yticks(fontsize=fontSize)
plt.xticks(fontsize=fontSize);

In [ ]:
percents, _, _ = plt.hist(X_val_r[~shared_mask], bins=3, range=(0, 3), density=True, edgecolor='black')
plt.xticks(np.arange(0, 3)+.5, ['Not Water', 'Open Surface Water', 'Partial Surface Water'],fontsize=9)
plt.ylabel('Percent',fontsize=9)
plt.title('For HLS pixels to be compared, class breakdown relative to Validation data');
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))

In [ ]:
plt.close('all')

In [ ]:
np.random.seed(0)
# list of lists, 
# each index corresponds to class label 0, 1, 2 and the list will contain
# indices within the flattend array of that label
stratified_selection = [] 
validation_hls_pixels = X_val_r[~shared_mask]
dswx_hls_pixels = X_dswx_c[~shared_mask]

TOTAL_SAMPLES = 500

for label in [0, 1, 2]:
    indices = np.argwhere(validation_hls_pixels == label).ravel()
    subset_size = int(np.ceil(percents[label] * TOTAL_SAMPLES))
    indices_subset = np.random.choice(indices, subset_size, replace=False)
    stratified_selection.append(indices_subset)


In [ ]:
validation_labels = [label for label in [0, 1, 2] for k in range(len(stratified_selection[label]))]
opera_dswx_labels = [dswx_label 
                     for label in [0, 1, 2] 
                     for dswx_label in dswx_hls_pixels[stratified_selection[label]]]
len(validation_labels), len(opera_dswx_labels)

In [ ]:
X_samples = np.full(shared_mask.shape, 0)
temp = X_samples[~shared_mask]

k = 1

sample2validation = {}
sample2dswx = {}

for label in [0, 1, 2]:
    for ind in stratified_selection[label]:
        temp[ind] = k
        # ensure we can look up the sample id and find the correct label for the different datasets we compare
        sample2validation[k] = label
        sample2dswx[k] = dswx_hls_pixels[ind]
        k += 1
    
X_samples[~shared_mask] = temp
(X_samples > 0).sum()

In [ ]:
features = get_geopandas_features_from_array(# Note 8 bits is not enough for 500 points
                                             X_samples.astype(np.int32), 
                                             transform=p_dswx_c['transform'],
                                             mask=(X_samples==0),
                                             label_name='sample_id'
                                            )
df_samples = gpd.GeoDataFrame.from_features(features, 
                                            crs=p_dswx_c['crs'])
df_samples['val_label'] = df_samples['sample_id'].map(lambda label: sample2validation[label])
df_samples['dswx_label'] = df_samples['sample_id'].map(lambda label: sample2dswx[label])

df_samples.head()

In [ ]:
fontSize=7
fontSizeTitles=8
fig, ax = plt.subplots(1, 2, dpi=150, figsize=(8, 8))
cmap = ListedColormap([np.array(colormap[key]) / 255 for key in range(256)])

show(X_dswx_c,cmap=cmap,transform=p_dswx_c['transform'],interpolation='none',ax=ax[0],vmin=0,vmax=255)
df_samples.to_crs(p_dswx_c['crs']).boundary.plot(ax=ax[0], color='magenta',alpha=1)
ax[0].set_title('Sampling points (magenta) and DSWx data',fontsize=fontSizeTitles)
ax[0].set_xlabel('UTM easting (meters)',fontsize=fontSizeTitles)
ax[0].set_ylabel('UTM northing (meters)',fontsize=fontSizeTitles)
ax[0].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax[0].tick_params(axis='both', which='major', labelsize=fontSize)
ax[0].yaxis.get_offset_text().set_fontsize(fontSize)
ax[0].xaxis.get_offset_text().set_fontsize(fontSize)

show(X_val_r, cmap=cmap, transform=p_val_r['transform'], interpolation='none',ax=ax[1], vmin=0, vmax=255)
df_samples.to_crs(p_val_r['crs']).boundary.plot(ax=ax[1], color='magenta',alpha=1)
ax[1].set_title('Sampling points (magenta) and Validation data',fontsize=fontSizeTitles)
ax[1].set_xlabel('UTM easting (meters)',fontsize=fontSize)
ax[1].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
ax[1].tick_params(axis='both', which='major', labelsize=fontSize)
ax[1].yaxis.get_offset_text().set_fontsize(fontSize)
ax[1].xaxis.get_offset_text().set_fontsize(fontSize)

In [ ]:
with rasterio.open(out_dir / f'samples_{DSWx_ID}.tif', 'w', **p_dswx_c) as ds:
    ds.write(X_samples, 1)

In [ ]:
df_samples.to_file(out_dir / f'samples_{DSWx_ID}')

# Accuracy Assessment

In [ ]:
class_dict = {0: 'Not_Water',
              1: 'Open_Surface_Water', 
              2: 'Partial_Surface_Water'}
Target_Names=['Not_Water','Open_Surface_Water','Partial_Surface_Water']

y_val = validation_hls_pixels
y_dswx = dswx_hls_pixels

## comment to inspect all pixels
y_val = np.array([label for ind in stratified_selection for label in validation_hls_pixels[ind]])
y_dswx = np.array([label  for ind in stratified_selection for label in dswx_hls_pixels[ind]])

## Total Accuracy

In [ ]:
# total_acc=(y_dswx == y_val).sum()/y_val.size
total_acc=sklearn.metrics.accuracy_score(y_val, y_dswx)
print('Total Accuracy = ', total_acc*100 ,'%')

## Confusion Matrix

In [ ]:
y_dswx_str = pd.Series([class_dict[class_id] for class_id in y_dswx], name='OPERA_DSWx')
y_val_str = pd.Series([class_dict[class_id] for class_id in y_val], name='OPERA_Validation')
df_conf = pd.crosstab(y_val_str, y_dswx_str)
df_conf

In [ ]:
df_conf_formatted = df_conf.astype(int)
name = df_conf.index.name
df_conf_formatted.rename(index={index: f'{index}_{name}' for index in df_conf.index}, inplace=True)
col_name = df_conf.columns.name
df_conf_formatted.rename(columns={col: f'{col}_{col_name}' for col in df_conf.columns}, inplace=True)
df_conf_formatted.to_dict();

## Classification Report

In [ ]:
prec, recall, f1, supp = sklearn.metrics.precision_recall_fscore_support(y_val, y_dswx, labels=[0, 1, 2])
recall_per_class = {class_dict[label]: recall[label] for label in [0, 1, 2]}
prec_per_class = {class_dict[label]: prec[label] for label in [0, 1, 2]}
f1_per_class = {class_dict[label]: f1[label] for label in [0, 1, 2]}
supp_per_class = {class_dict[label]: int(supp[label]) for label in [0, 1, 2]}

In [ ]:
Class_Report = sklearn.metrics.classification_report(y_val, y_dswx, labels=[0, 1, 2], target_names=Target_Names)
print(Class_Report)

## Accuracy per class

In [ ]:
acc_per_class = {}

for c in [0, 1, 2]:
    y_val_temp = y_val.copy()
    y_dswx_temp = y_dswx.copy()
    
    y_val_temp[y_val_temp != c] = 255
    y_dswx_temp[y_dswx_temp != c] = 255

    acc_per_class[class_dict[c]] = (y_val_temp == y_dswx_temp).sum() / y_dswx.size

acc_per_class

## Serialize

In [ ]:
assessment = {'dswx_id': DSWx_ID,
              'precision': prec_per_class,
              'recall': recall_per_class,
              'f1_per_class': f1_per_class,
              'supp_per_class':supp_per_class,
              'confusion_matrix': df_conf_formatted.to_dict(),
              'accuracy_per_class': acc_per_class}

json.dump(assessment, open(out_dir / f'assessment_{DSWx_ID}.json', 'w'))

In [ ]:
df_conf.to_csv(out_dir / f'confusion_matrix_{DSWx_ID}.csv')

# Verify Requirements

In [ ]:
open_surface_water_req = False
if assessment['accuracy_per_class']['Open_Surface_Water'] >= .80:
    open_surface_water_req = True

In [ ]:
partial_surface_water_req = False
if assessment['accuracy_per_class']['Partial_Surface_Water'] >= .70:
    partial_surface_water_req = True

In [ ]:
print(f'Passed Open Surface Water Requirement: ', open_surface_water_req)
print(f'Passed Partial Surface Water Requirement: ', partial_surface_water_req)

In [ ]:
hls_urls_dict = get_hls_urls(HLS_ID)

In [ ]:
dswx_urls_dict = {f'dswx-{url.split(".")[-2][6:]}': url 
                  for url in dswx_urls}

In [ ]:
requirement_verification = {'dswx_id': DSWx_ID,
                            'surface_water': open_surface_water_req,
                            'partial_surface_water': partial_surface_water_req,
                            **dswx_urls_dict,
                            'planet_id': PLANET_ID,
                            'validation_dataset_s3': get_val_s3_path(PLANET_ID),
                            'hls_id': HLS_ID,
                            **hls_urls_dict}

json.dump(requirement_verification, open(out_dir / f'requirement_verification_{DSWx_ID}.json', 'w'))

Lastly, calculate differance map to show where DSWx and Validation data agree/disagree

In [ ]:
X_diff_temp=X_dswx_c-X_val_r
X_diff=X_diff_temp

X_diff=X_diff_temp.astype('float32')
X_diff[(X_diff_temp < -5) | (X_diff_temp > 5)] = np.nan

print(["min = "  + str(np.nanmin(X_diff)), "max = " + str(np.nanmax(X_diff))])

In [ ]:
fontSize=7
fontSizeTitle=8
fig, axs = plt.subplots(1, 3, dpi=150, figsize=(10, 10))
cmap = ListedColormap([np.array(colormap[key]) / 255 for key in range(256)])

im0 = axs[0].imshow(X_dswx_c, cmap=cmap, vmin=0, interpolation='none', vmax=255)
show(X_dswx_c, interpolation='none',cmap=cmap,vmin=0,vmax=255,transform=p_dswx_c['transform'],ax=axs[0])
axs[0].set_title('DSWx',fontsize=fontSizeTitle)
axs[0].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
axs[0].set_xlabel('UTM easting (meters)',fontsize=fontSize)
axs[0].set_ylabel('UTM northing (meters)',fontsize=fontSize)
axs[0].yaxis.get_offset_text().set_fontsize(fontSize)
axs[0].xaxis.get_offset_text().set_fontsize(fontSize)
cbar=fig.colorbar(im0,ax=axs[0],shrink=0.2,ticks=[0.5, 1.5, 2.5,5.5,8.5, 9.5])
cbar.set_ticklabels(['Not Water', 'Open Water','Partial Surface Water','Reserved','Snow/Ice','Cloud/Cloud Shadow'],fontsize=5,rotation=0)   
cbar.ax.set_ylim(0,10)
cbar.ax.tick_params(labelsize=5)

im1 = axs[1].imshow(X_val_r, cmap=cmap, vmin=0, interpolation='none', vmax=255)
show(X_val_r, interpolation='none',cmap=cmap,vmin=0,vmax=255,transform=p_val_r['transform'],ax=axs[1])
axs[1].set_title('Validation data',fontsize=fontSizeTitle)
axs[1].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
axs[1].set_xlabel('UTM easting (meters)',fontsize=fontSize)
axs[1].yaxis.get_offset_text().set_fontsize(fontSize)
axs[1].xaxis.get_offset_text().set_fontsize(fontSize)
cbar=fig.colorbar(im1,ax=axs[1],shrink=0.2,ticks=[0.5, 1.5, 2.5,5.5,8.5, 9.5])
cbar.set_ticklabels(['Not Water', 'Open Water','Partial Surface Water','Reserved','Snow/Ice','Cloud/Cloud Shadow'],fontsize=5)   
cbar.ax.set_ylim(0,10)
cbar.ax.tick_params(labelsize=5)

cmapDiff=ListedColormap([(51/255,0/255, 102/255), (7/255, 137/255, 66/255), (1, 1, 1), (0,0,0),(231/255,41/255,138/255)])
im2 = axs[2].imshow(X_diff, interpolation='none',vmin=-2, vmax=2,cmap=cmapDiff)
show(X_diff, interpolation='none',vmin=-2, vmax=2,transform=p_val_r['transform'],ax=axs[2],cmap=cmapDiff)
axs[2].set_title('Difference Map (DSWx - Val.)',fontsize=fontSizeTitle)
axs[2].set_xlabel('UTM easting (meters)',fontsize=fontSize)
axs[2].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
axs[2].yaxis.get_offset_text().set_fontsize(fontSize)
axs[2].xaxis.get_offset_text().set_fontsize(fontSize)
cbar=fig.colorbar(im2,ax=axs[2],ticks=[-2,-1,0,1,2],shrink=0.2)
cbar.ax.tick_params(labelsize=6)
cbar.ax.set_ylim(-2,2)
cbar.ax.tick_params(labelsize=5)
im2.set_cmap(cmapDiff);
cmapDiff.set_bad(color='gray')



In [ ]:
cmapDiffSave=({-2: (51,0, 102, 255),
 -1: (7, 137, 66, 255),
 0: (255, 255, 255, 255),
 1: (0,0,0, 255),
 2: (231,41,138, 255)})

with rasterio.open(out_dir / f'DifferenceMap.tif', 'w', **p_dswx_c) as ds:
    ds.write(X_diff, 1)
    ds.write_colormap(1, cmapDiffSave)